#Question 2

In [9]:
import os
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import pickle

In [10]:
#IMPORTING TEXT DATA...
Training_Folder = 'labeled_articles/'
Training_Label = []
Training_Matrix = []
for file in os.listdir(Training_Folder):
    #print (file)
    Text_File = open(Training_Folder+file,'r+',encoding='utf-8')
    for line in Text_File:
        if (line.startswith('###')):
            continue
#### Sentence preprocessing for cleaning
        line = re.sub(r'[-*+%$()\.,/?!><"&#\[\]\(\)\\]', ' ',line)
        lines = line.split()
        #print(lines)
#### Putting lines into the list for classification
        Training_Matrix.append(lines[1:])
#### Making file name as label for classification
        Training_Label.append(lines[0])
        #print(Training_Label)
    Text_File.close()

In [11]:
#Count vectorizer...
vec=CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)
tm=vec.fit_transform(Training_Matrix)

In [13]:
#Label Encoder
le=LabelEncoder()
le.fit(Training_Label)
tl=le.transform(Training_Label)

In [14]:
#Splitting train and test data using stratified split
X_train, X_test, Y_train, Y_test = train_test_split(tm , tl, test_size=0.3, random_state = 2000)
# print(Y_test.shape)

In [15]:
#K fold cross validation
kf = KFold(n_splits=4)
kf.get_n_splits(tm)
for train_index, test_index in kf.split(X_train):
    #print(train_index,test_index)
    X_trainval, X_testval = X_train[train_index],X_train[test_index]
    Y_trainval, Y_testval = Y_train[train_index],Y_train[test_index]
    #print(Y_train.shape)
    #print(X_train.shape)

In [16]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(),
    SGDClassifier(),
    OneVsRestClassifier(LinearSVC()),
    RandomForestClassifier(),
    MultinomialNB(),
    DecisionTreeClassifier()]

In [17]:
print("***After cross validation...***")
max_accuracy=0
for clf in classifiers:
    clf.fit(X_trainval,Y_trainval)
    
    name = clf.__class__.__name__
    print("***",name,"***")
    predictions = clf.predict(X_testval)
    #Train accuracy
    ac_tr=accuracy_score(Y_trainval,clf.predict(X_trainval))
    print("Training accuracy:",ac_tr*100,"%")
    #Validation accuracy
   
    ac_val=accuracy_score(Y_testval, predictions)
    if ac_val>max_accuracy:#Remembering model with highest accuracy
        max_accuracy=ac_val
        s = pickle.dumps(clf)
    print("Validation score:",ac_val*100,"%")
    print(confusion_matrix(Y_test,predictions))
    print(classification_report(Y_test,predictions))

***After cross validation...***
*** KNeighborsClassifier ***
Training accuracy: 86.36919315403424 %
Validation score: 67.70642201834862 %


ValueError: Found input variables with inconsistent numbers of samples: [936, 545]

In [ ]:
#Using saved model for testing
s = pickle.dumps(clf)
clf2 = pickle.loads(s)
name = clf2.__class__.__name__
print("***Saved classifier is",name,"***")
pred2=clf2.predict(X_test)
ac_tes=accuracy_score(Y_test, pred2)
print("Model accuracy:",ac_tes*100,"%")